<center><H1>CREACIÓN DE UNA BASE DE DATOS ANALÍTICA</H1></center>

<H1>Autor: Felipe Fernández Rodriguez</H1>

<h3>Problema</h3>
<p>El Hospital San Juan de Dios de Rionegro enfrenta el reto de optimizar la gestión de sus recursos hospitalarios, especialmente en lo relacionado con la distribución de camas, la asignación de personal médico y la atención oportuna de pacientes. En la actualidad, gran parte del seguimiento operativo se realiza de forma manual o mediante reportes aislados, lo que dificulta identificar tendencias sobre la ocupación de camas, los niveles de satisfacción de los pacientes, la carga laboral del personal y la eficiencia de los distintos servicios hospitalarios.

Ante esta situación, surge la necesidad de implementar una base de datos analítica que permita integrar información de los diferentes procesos hospitalarios —como el registro de pacientes, el control de camas disponibles, la programación del personal y los indicadores de servicio— para facilitar el análisis de datos históricos y apoyar la toma de decisiones basada en evidencia.

Con este sistema analítico, el Hospital San Juan de Dios de Rionegro podría, por ejemplo, anticipar la demanda de servicios, equilibrar la carga laboral del personal, mejorar la asignación de recursos y monitorear el nivel de satisfacción de los pacientes. En conjunto, esto permitiría incrementar la eficiencia operativa y la calidad del servicio médico ofrecido a la comunidad.</p>

<h3>Dataset</h3>
<ul>
  <li>Nombre: Hospital Beds Management</li>
  <li>URL: https://www.kaggle.com/datasets/jaderz/hospital-beds-management/</li>
</ul>

<h3>¿Porqué se seleccionó el dataset "Hospital Beds Management"?</h3>
<p>El dataset “Hospital Beds Management” fue seleccionado por varias razones que lo convierten en una excelente fuente de información para el desarrollo de un análisis académico en el contexto del Hospital San Juan de Dios de Rionegro.

En primer lugar, el conjunto de datos cuenta con una evaluación de usabilidad completa (100%) en los criterios de Completitud, Credibilidad y Compatibilidad, lo cual garantiza que la información contenida es coherente, bien estructurada y apta para análisis. Además, su licencia CC0: Public Domain permite el uso libre de los datos con fines educativos y de investigación, lo que resulta ideal para el propósito de este trabajo universitario.

Otro factor relevante es que el dataset está compuesto por cuatro archivos CSV (patients.csv, services_weekly.csv, staff.csv y staff_schedule.csv) que en conjunto tienen un peso de 410.15 KB, proporcionando una cantidad considerable de registros para realizar análisis estadísticos, validaciones y simulaciones propias del ámbito de Big Data. Cada archivo aborda un componente esencial de la operación hospitalaria —pacientes, servicios, personal médico y horarios—, lo que facilita un enfoque integral del problema de gestión hospitalaria.

Asimismo, el dataset presenta cero valores faltantes (Missing 0%) y un 100% de datos válidos (Valid 100%), garantizando su calidad y confiabilidad para el procesamiento y análisis.

Finalmente, el hecho de que los datos sean sintéticos y generados para fines educativos y analíticos permite trabajar con escenarios realistas sin comprometer la privacidad ni la confidencialidad de personas reales. Esto ofrece un entorno controlado y ético para aplicar técnicas de análisis de datos, modelado y simulación orientadas a optimizar la gestión de camas, personal y servicios dentro del hospital.

En resumen, este dataset fue seleccionado por su alta calidad técnica, licencia abierta, estructura completa, ausencia de datos faltantes y pertinencia temática, factores que lo hacen adecuado para realizar un estudio analítico aplicado al contexto hospitalario real del Hospital San Juan de Dios de Rionegro.</p>

<h3>Descripción de las variables relevantes y su utilidad</h3>
<p>El dataset utilizado para este proyecto contiene información simulada de la operación hospitalaria, organizada en cuatro archivos principales: patients.csv, services_weekly.csv, staff.csv y staff_schedule.csv. Cada archivo aporta variables clave que permiten analizar diferentes aspectos del funcionamiento del Hospital San Juan de Dios de Rionegro, desde la atención de pacientes hasta la gestión del personal y los recursos disponibles.</p>
<ol>
  <li>Archivo: patients.csv:
    <ul>
      <li>patient_id: Identificador único de cada paciente, útil para realizar un seguimiento individual de los casos y relacionar los registros con otros conjuntos de datos.</li>
    </ul>
    <ul>
      <li>name: Nombre completo del paciente. Se usa solo como referencia descriptiva, ya que el análisis se centra en variables numéricas y categóricas.</li>
    </ul>
    <ul>
      <li>age: Edad del paciente. Permite realizar análisis demográficos para identificar qué grupos etarios requieren más atención o presentan mayores tasas de hospitalización.</li>
    </ul>
    <ul>
      <li>arrival_date / departure_date: Fechas de ingreso y egreso hospitalario. Son útiles para calcular la duración promedio de las estancias, analizar la ocupación de camas y determinar la rotación de pacientes por servicio.</li>
    </ul>
    <ul>
      <li>service: Servicio hospitalario asignado (por ejemplo, cirugía, medicina general o urgencias). Facilita la segmentación de los datos y el análisis de la demanda por área médica.</li>
    </ul>
    <ul>
      <li>satisfaction: Puntuación de satisfacción del paciente (0–100). Indicador esencial para medir la calidad del servicio percibida y relacionarla con la carga operativa o los recursos disponibles.</li>
    </ul>
  </li>
  <li>Archivo: services_weekly.csv
    <ul>
      <li>week / month: Permiten la organización temporal del análisis y la creación de series históricas para observar tendencias o estacionalidades.</li>
    </ul>
    <ul>
      <li>service: Servicio hospitalario al que pertenece el registro, usado para relacionar los datos con otros conjuntos (pacientes y personal).</li>
    </ul>
    <ul>
      <li>available_beds: Número de camas disponibles. Es una variable clave para medir la capacidad hospitalaria y su utilización.</li>
    </ul>
    <ul>
      <li>patients_request: Número de pacientes que solicitaron ingreso durante la semana. Permite estimar la demanda hospitalaria.</li>
    </ul>
    <ul>
      <li>patients_admitted: Número de pacientes admitidos efectivamente. En conjunto con las anteriores variables, sirve para calcular la tasa de ocupación y rechazo.</li>
    </ul>
    <ul>
      <li>patients_refused: Pacientes que no pudieron ser atendidos por falta de capacidad. Este indicador refleja posibles cuellos de botella o saturación en los servicios.</li>
    </ul>
    <ul>
      <li>patient_satisfaction: Satisfacción promedio semanal de los pacientes atendidos en el servicio, útil para evaluar la relación entre la satisfacción y la carga operativa.</li>
    </ul>
    <ul>
      <li>staff_morale: Puntuación promedio de motivación del personal (0–100). Indicador importante para estudiar la relación entre la moral del personal y los resultados del servicio.</li>
    </ul>
    <ul>
      <li>event: Describe eventos especiales (como brotes de enfermedades o temporadas de alta demanda), útiles para analizar su impacto en la operación hospitalaria.</li>
    </ul>
  </li>
  <li>Archivo: staff.csv
    <ul>
      <li>staff_id: Identificador único de cada miembro del personal. Permite relacionar los registros de personal con los datos de asistencia semanal.</li>
    </ul>
    <ul>
      <li>staff_name: Nombre completo del empleado. Se usa principalmente con fines descriptivos.</li>
    </ul>
    <ul>
      <li>role: Rol desempeñado (por ejemplo, médico, enfermero, auxiliar). Es esencial para analizar la distribución del personal por tipo de función y evaluar la disponibilidad de recursos humanos.</li>
    </ul>
    <ul>
      <li>service: Servicio donde trabaja el personal. Facilita el análisis de la dotación de personal por área hospitalaria.</li>
    </ul>
  </li>
  <li>Archivo: staff_schedule.csv
    <ul>
      <li>week: Semana del año a la que corresponde el registro. Permite analizar la asistencia y rotación del personal a lo largo del tiempo.</li>
    </ul>
    <ul>
      <li>staff_id: Identificador del empleado. Se relaciona directamente con el archivo staff.csv para integrar la información personal y su rol.</li>
    </ul>
    <ul>
      <li>staff_name, role, service: Atributos descriptivos que complementan la información del empleado en cada registro semanal.</li>
    </ul>
    <ul>
      <li>present: Variable binaria (1 = presente, 0 = ausente) que indica si el empleado trabajó esa semana. Es fundamental para analizar la presencia del personal y su impacto en la atención de pacientes y la moral del equipo.</li>
    </ul>
  </li>
</ol>

### Modelo Entidad–Relación (ERD)

El siguiente diagrama representa el modelo entidad–relación diseñado a partir del dataset *Hospital Beds Management* para el Hospital San Juan de Dios de Rionegro.

![Modelo ERD](https://github.com/felipefernandezr-datascience/BigData/raw/main/images/ERD.png?raw=true)

<h3>Creación BD Y Carga de Datos</h3>

In [0]:
# Importa SparkSession, que permite crear la sesión de trabajo con Spark
from pyspark.sql import SparkSession

In [0]:
# Crea o obtiene una sesión activa de Spark para trabajar con datos
spark = SparkSession.builder.getOrCreate()

<h3>Carga de datos patients.csv</h3>

In [0]:
# Lee el archivo CSV de patients y lo carga en un DataFrame de Spark
patients_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/patients.csv")


In [0]:
# Muestra la estructura del DataFrame, incluyendo los nombres de las columnas y sus tipos de datos
patients_df.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- service: string (nullable = true)
 |-- satisfaction: integer (nullable = true)



In [0]:
# Muestra las primeras 5 filas del DataFrame completas, sin cortar el contenido de las columnas
patients_df.show(5, truncate = False)

+------------+-----------------+---+------------+--------------+----------------+------------+
|patient_id  |name             |age|arrival_date|departure_date|service         |satisfaction|
+------------+-----------------+---+------------+--------------+----------------+------------+
|PAT-09484753|Richard Rodriguez|24 |2025-03-16  |2025-03-22    |surgery         |61          |
|PAT-f0644084|Shannon Walker   |6  |2025-12-13  |2025-12-14    |surgery         |83          |
|PAT-ac6162e4|Julia Torres     |24 |2025-06-29  |2025-07-05    |general_medicine|83          |
|PAT-3dda2bb5|Crystal Johnson  |32 |2025-10-12  |2025-10-23    |emergency       |81          |
|PAT-08591375|Garrett Lin      |25 |2025-02-18  |2025-02-25    |ICU             |76          |
+------------+-----------------+---+------------+--------------+----------------+------------+
only showing top 5 rows


In [0]:
# Crea o reemplaza una vista temporal llamada 'view_patients' para poder consultar el DataFrame con SQL
patients_df.createOrReplaceTempView('view_patients')

In [0]:
%sql
-- Consulta SQL que muestra las primeras 5 filas de la vista temporal 'view_patients'
SELECT * FROM view_patients LIMIT 5

patient_id,name,age,arrival_date,departure_date,service,satisfaction
PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61
PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83
PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83
PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81
PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76


In [0]:
# Guarda el DataFrame como una tabla permanente en el esquema 'hospital_beds_db'
patients_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_patients')

<h3>Consultas SQL tabla tbl_patients</h3>

In [0]:
%sql
-- Consulta SQL que cuenta el número total de registros almacenados en la tabla 'tbl_patients'
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_patients


COUNT(*)
1000


<p>La tabla tbl_patients contiene un total de 1000 registros, lo que indica que en el conjunto de datos se almacenan los datos de mil pacientes atendidos en el hospital.</p>

In [0]:
%sql
-- Muestra la estructura de la tabla 'tbl_patients', incluyendo los nombres de las columnas, tipos de datos y comentarios
DESCRIBE TABLE workspace.hospital_beds_db.tbl_patients

col_name,data_type,comment
patient_id,string,null
name,string,null
age,int,null
arrival_date,date,null
departure_date,date,null
service,string,null
satisfaction,int,null


<p>La estructura de la tabla muestra los campos principales asociados a la gestión de pacientes, incluyendo su identificación, datos personales, fechas de ingreso y salida, el servicio hospitalario recibido y el nivel de satisfacción. Los tipos de datos son coherentes con la naturaleza de la información que se almacena.</p>

In [0]:
%sql
-- Consulta SQL que muestra los 5 pacientes con mayor nivel de satisfacción (satisfaction)
-- del servicio de urgencias ('emergency'), ordenados de mayor a menor puntuación
SELECT satisfaction 
FROM workspace.hospital_beds_db.tbl_patients
WHERE service = 'emergency'
ORDER BY satisfaction DESC
LIMIT 5

satisfaction
99
99
99
99
99


<p>Los cinco pacientes con mayor nivel de satisfacción atendidos en el servicio de emergencias calificaron con 99 puntos, lo que evidencia un alto grado de satisfacción en este servicio, al menos entre los registros con las puntuaciones más elevadas.</p>

<h3>Carga de datos services_weekly.csv</h3>

In [0]:
# Lee el archivo CSV de services_weekly y lo carga en un DataFrame de Spark
services_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/services_weekly.csv")

In [0]:
# Muestra la estructura del DataFrame, incluyendo los nombres de las columnas y sus tipos de datos
services_df.printSchema()

root
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- available_beds: integer (nullable = true)
 |-- patients_request: integer (nullable = true)
 |-- patients_admitted: integer (nullable = true)
 |-- patients_refused: integer (nullable = true)
 |-- patient_satisfaction: integer (nullable = true)
 |-- staff_morale: integer (nullable = true)
 |-- event: string (nullable = true)



In [0]:
# Muestra las primeras 5 filas del DataFrame completas, sin cortar el contenido de las columnas
services_df.show(5, truncate = False)

+----+-----+----------------+--------------+----------------+-----------------+----------------+--------------------+------------+-----+
|week|month|service         |available_beds|patients_request|patients_admitted|patients_refused|patient_satisfaction|staff_morale|event|
+----+-----+----------------+--------------+----------------+-----------------+----------------+--------------------+------------+-----+
|1   |1    |emergency       |32            |76              |32               |44              |67                  |70          |none |
|1   |1    |surgery         |45            |130             |45               |85              |83                  |78          |flu  |
|1   |1    |general_medicine|37            |201             |37               |164             |97                  |43          |flu  |
|1   |1    |ICU             |22            |31              |22               |9               |84                  |91          |flu  |
|2   |1    |emergency       |28          

In [0]:
# Crea o reemplaza una vista temporal llamada 'view_services' para poder consultar el DataFrame con SQL
services_df.createOrReplaceTempView('view_services')

In [0]:
%sql
-- Consulta SQL que muestra las primeras 5 filas de la vista temporal 'view_services'
SELECT * FROM view_services LIMIT 5

week,month,service,available_beds,patients_request,patients_admitted,patients_refused,patient_satisfaction,staff_morale,event
1,1,emergency,32,76,32,44,67,70,none
1,1,surgery,45,130,45,85,83,78,flu
1,1,general_medicine,37,201,37,164,97,43,flu
1,1,ICU,22,31,22,9,84,91,flu
2,1,emergency,28,169,28,141,75,64,none


In [0]:
# Guarda el DataFrame como una tabla permanente en el esquema 'hospital_beds_db'
services_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_services')

<h3>Consultas SQL tabla tbl_services</h3>

In [0]:
%sql
-- Consulta SQL que cuenta el número total de registros almacenados en la tabla 'tbl_services'
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_services

COUNT(*)
208


<p>La tabla tbl_services contiene 208 registros, lo que sugiere que el conjunto de datos almacena información semanal o mensual sobre los distintos servicios del hospital, incluyendo su disponibilidad y desempeño operativo.</p>

In [0]:
%sql
-- Muestra la estructura de la tabla 'tbl_services', incluyendo los nombres de las columnas, tipos de datos y comentarios
DESCRIBE TABLE workspace.hospital_beds_db.tbl_services

col_name,data_type,comment
week,int,null
month,int,null
service,string,null
available_beds,int,null
patients_request,int,null
patients_admitted,int,null
patients_refused,int,null
patient_satisfaction,int,null
staff_morale,int,null
event,string,null


<p>La estructura de la tabla tbl_services refleja variables relacionadas con la gestión hospitalaria por servicio, incluyendo la disponibilidad de camas, la demanda de pacientes, las admisiones, rechazos, satisfacción de los pacientes, moral del personal y posibles eventos relevantes. Esto permite analizar tanto el rendimiento operativo como la calidad del servicio.</p>

In [0]:
%sql
-- Consulta SQL que muestra los 5 registros del servicio de urgencias ('emergency')
-- con mayor número de pacientes admitidos (patients_admitted),
-- junto con la cantidad de pacientes rechazados (patients_refused),
-- ordenados de mayor a menor número de admitidos
SELECT patients_admitted, patients_refused
FROM workspace.hospital_beds_db.tbl_services
WHERE service = 'emergency'
ORDER BY patients_admitted DESC
LIMIT 5

patients_admitted,patients_refused
36,151
34,164
33,94
32,145
32,44


<p>En el servicio de emergencias, se observa que aunque el número de pacientes admitidos es relativamente constante (entre 32 y 36), el número de pacientes rechazados varía significativamente, alcanzando hasta 164 rechazos en algunos periodos. Esto puede indicar limitaciones en la capacidad de atención o alta demanda en ciertos momentos.</p>

<h3>Carga de datos staff.csv</h3>

In [0]:
# Lee el archivo CSV de staff y lo carga en un DataFrame de Spark
staff_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/staff.csv")

In [0]:
# Muestra la estructura del DataFrame, incluyendo los nombres de las columnas y sus tipos de datos
staff_df.printSchema()

root
 |-- staff_id: string (nullable = true)
 |-- staff_name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- service: string (nullable = true)



In [0]:
# Muestra las primeras 5 filas del DataFrame completas, sin cortar el contenido de las columnas
staff_df.show(5, truncate = False)

+------------+---------------+------+---------+
|staff_id    |staff_name     |role  |service  |
+------------+---------------+------+---------+
|STF-5ca26577|Allison Hill   |doctor|emergency|
|STF-02ae59ca|Noah Rhodes    |doctor|emergency|
|STF-d8006e7c|Angie Henderson|doctor|emergency|
|STF-212d8b31|Daniel Wagner  |doctor|emergency|
|STF-107a58e4|Cristian Santos|doctor|emergency|
+------------+---------------+------+---------+
only showing top 5 rows


In [0]:
# Crea o reemplaza una vista temporal llamada 'view_staff' para poder consultar el DataFrame con SQL
staff_df.createOrReplaceTempView('view_staff')

In [0]:
%sql
-- Consulta SQL que muestra las primeras 5 filas de la vista temporal 'view_staff'
SELECT * FROM view_staff LIMIT 5

staff_id,staff_name,role,service
STF-5ca26577,Allison Hill,doctor,emergency
STF-02ae59ca,Noah Rhodes,doctor,emergency
STF-d8006e7c,Angie Henderson,doctor,emergency
STF-212d8b31,Daniel Wagner,doctor,emergency
STF-107a58e4,Cristian Santos,doctor,emergency


In [0]:
# Guarda el DataFrame como una tabla permanente en el esquema 'hospital_beds_db'
staff_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_staff')

<h3>Consultas SQL tabla tbl_staff</h3>

In [0]:
%sql
-- Consulta SQL que cuenta el número total de registros almacenados en la tabla 'tbl_staff'
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_staff

COUNT(*)
110


<p>La tabla tbl_staff contiene 110 registros, lo que representa el número total de empleados registrados en el hospital. Este dato permite dimensionar la capacidad del personal disponible para la atención médica y los diferentes servicios hospitalarios.</p>

In [0]:
%sql
-- Muestra la estructura de la tabla 'tbl_staff', incluyendo los nombres de las columnas, tipos de datos y comentarios
DESCRIBE TABLE workspace.hospital_beds_db.tbl_staff

col_name,data_type,comment
staff_id,string,null
staff_name,string,null
role,string,null
service,string,null


<p>La estructura de la tabla tbl_staff describe la información básica del personal hospitalario, incluyendo un identificador único, el nombre del empleado, su rol dentro del hospital y el servicio en el que trabaja. Esta información es esencial para la asignación de recursos humanos y la gestión de turnos y servicios.</p>

In [0]:
%sql
-- Consulta SQL que obtiene los diferentes roles del personal hospitalario 
-- registrados en la tabla 'tbl_staff', evitando duplicados.
SELECT DISTINCT role
FROM workspace.hospital_beds_db.tbl_staff

role
doctor
nurse
nursing_assistant


<p>El personal del hospital está conformado por tres roles principales: doctores, enfermeros y auxiliares de enfermería. Esta distribución refleja una estructura jerárquica y funcional típica en entornos hospitalarios, donde cada rol cumple tareas complementarias para garantizar una atención médica integral.</p>

<h3>Carga de datos staff_schedule.csv</h3>

In [0]:
# Lee el archivo CSV de staff_schedule y lo carga en un DataFrame de Spark
schedule_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/staff_schedule.csv")

In [0]:
# Muestra la estructura del DataFrame, incluyendo los nombres de las columnas y sus tipos de datos
schedule_df.printSchema()

root
 |-- week: integer (nullable = true)
 |-- staff_id: string (nullable = true)
 |-- staff_name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- service: string (nullable = true)
 |-- present: integer (nullable = true)



In [0]:
# Muestra las primeras 5 filas del DataFrame completas, sin cortar el contenido de las columnas
schedule_df.show(5, truncate = False)

+----+------------+------------+------+---------+-------+
|week|staff_id    |staff_name  |role  |service  |present|
+----+------------+------------+------+---------+-------+
|1   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
|2   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
|3   |STF-b77cdc60|Allison Hill|doctor|emergency|0      |
|4   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
|5   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
+----+------------+------------+------+---------+-------+
only showing top 5 rows


In [0]:
# Crea o reemplaza una vista temporal llamada 'view_schedule' para poder consultar el DataFrame con SQL
schedule_df.createOrReplaceTempView('view_schedule')

In [0]:
%sql
-- Consulta SQL que muestra las primeras 5 filas de la vista temporal 'view_schedule'
SELECT * FROM view_schedule LIMIT 5

week,staff_id,staff_name,role,service,present
1,STF-b77cdc60,Allison Hill,doctor,emergency,1
2,STF-b77cdc60,Allison Hill,doctor,emergency,1
3,STF-b77cdc60,Allison Hill,doctor,emergency,0
4,STF-b77cdc60,Allison Hill,doctor,emergency,1
5,STF-b77cdc60,Allison Hill,doctor,emergency,1


In [0]:
# Guarda el DataFrame como una tabla permanente en el esquema 'hospital_beds_db'
schedule_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_schedule')

<h3>Consultas SQL tabla tbl_schedule</h3>

In [0]:
%sql
-- Consulta SQL que cuenta el número total de registros almacenados en la tabla 'tbl_schedule'
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_schedule

COUNT(*)
6552


<p>La tabla tbl_schedule contiene 6.552 registros, lo que indica una amplia cantidad de datos relacionados con la programación y asistencia del personal hospitalario. Este volumen de información refleja la planificación continua del personal a lo largo de diferentes semanas y servicios dentro del hospital.</p>

In [0]:
%sql
-- Muestra la estructura de la tabla 'tbl_schedule', incluyendo los nombres de las columnas, tipos de datos y comentarios
DESCRIBE TABLE workspace.hospital_beds_db.tbl_schedule

col_name,data_type,comment
week,int,null
staff_id,string,null
staff_name,string,null
role,string,null
service,string,null
present,int,null


<p>La estructura de la tabla tbl_schedule muestra los campos relacionados con la planificación semanal del personal. Incluye el número de semana, la identificación y nombre del empleado, su rol, el servicio al que pertenece y un indicador de presencia laboral. Estos datos permiten analizar la distribución de turnos, la asistencia y la asignación de recursos humanos por servicio.</p>

In [0]:
%sql
-- Consulta SQL que muestra los diferentes servicios hospitalarios 
-- registrados en la tabla 'tbl_schedule', sin repetir valores.
SELECT DISTINCT service
FROM workspace.hospital_beds_db.tbl_schedule

service
emergency
surgery
general_medicine
ICU


<p>Esta consulta permite identificar los diferentes servicios hospitalarios en los que el personal está programado. Con esta información se puede determinar la cobertura del personal por área, garantizando que cada servicio del hospital cuente con los recursos humanos necesarios para su correcto funcionamiento.</p>